In [12]:
import pandas as pd
from scipy.special import binom
from tqdm import tqdm
from itertools import combinations
from pathlib import Path
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np


def rec_dd():
    return defaultdict(rec_dd)


In [13]:
benchmark_path = Path(
    "/home/gabrielathenes/Documents/study/HILARy/benchmark/partis_dataset"
)


In [14]:
def pairwise_evaluation(df, partition, ground_truth="simulations"):
    TP = 0
    P = binom(df.groupby([ground_truth]).size(), 2).sum()
    TP_FP = binom(df.groupby([partition]).size(), 2).sum()
    for _, family in tqdm(df.groupby([ground_truth]), disable=True):
        for r1, r2 in combinations(family[partition], 2):
            if r1 == r2:
                TP += 1
    if TP_FP == 0:
        return 0.0, 1.0
    elif P == 0:
        return None, None
    return (
        TP / P,
        TP / TP_FP,
    )  # sensitivity, precision

def entropy(dfGrouped):
    fs = dfGrouped.size()
    fs = fs / sum(fs)
    return sum(fs * np.log2(fs))


def variation_of_info(df, result):
    VI = (
        entropy(df.groupby(["simulations"]))
        + entropy(df.groupby([result]))
        - 2 * entropy(df.groupby([result, "simulations"]))
    )
    return VI


In [15]:
metric_dict=rec_dd()
for seed in [0, 1, 2]:
    prec_list=[]
    sens_list=[]
    var_list=[]
    for freq in ["0.01","0.05","0.10","0.20","0.30"]:
        dataframe=pd.DataFrame()
        for model in ["simulations","partis","scoper","hilary-1-1-7"]:
            if model=="scoper":
                df = pd.read_csv(benchmark_path / Path(f"seed-{seed}/scratch-mute-freq-{freq}/{model}/both_chains/partition.tsv"),
                    sep="\t",
                    index_col="sequence_id",
                )
            else :
                df = pd.read_csv(
                    [
                        each
                        for each in (benchmark_path
                        / Path(
                            f"seed-{seed}/scratch-mute-freq-{freq}/{model}/single_chain/"
                        )).glob("*igh.tsv")
                    ][0],
                    sep="\t",
                    index_col="sequence_id",
                )
            if model.startswith("hilary"):
                dataframe[model]=df["family"]
                dataframe["cdr3_length"]=df["cdr3_length"]
                dataframe[model+"_cdr3-based"]=df["cdr3_only_method_clustering"]
            else :
                dataframe[model]=df["clone_id"]

            sensitivity,precision=pairwise_evaluation(dataframe,model)
            varinfo = variation_of_info(dataframe,model)
            metric_dict[model]["precision"][freq][seed]=precision
            metric_dict[model]["sensitivity"][freq][seed]=sensitivity
            metric_dict[model]["variation_of_info"][freq][seed]=varinfo
            if model.startswith("hilary"):
                model_cdr3 = model + "_cdr3-based"
                sensitivity,precision=pairwise_evaluation(dataframe,model_cdr3)
                varinfo = variation_of_info(dataframe,model_cdr3)
                metric_dict[model_cdr3]["precision"][freq][seed]=precision
                metric_dict[model_cdr3]["sensitivity"][freq][seed]=sensitivity
                metric_dict[model_cdr3]["variation_of_info"][freq][seed]=varinfo


/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/tmp/ipykernel_523664/1910703448.py:15: DtypeWarning: Columns (2,3,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.re

In [16]:
print(metric_dict.keys())


dict_keys(['simulations', 'partis', 'scoper', 'hilary-1-1-7', 'hilary-1-1-7_cdr3-based'])


scoper
precision
sensitivity
variation_of_info
hilary-1-1-7_cdr3-based
precision
sensitivity
variation_of_info
hilary-1-1-7
precision
sensitivity
variation_of_info
partis
precision
sensitivity
variation_of_info


In [17]:
ls = [0.01,0.05,0.10,0.20,0.30]
fig = plt.figure(figsize=(14, 3))
ax1 = fig.add_subplot(131)
ax1.set_ylabel(r"Precision $\pi$")
ax1.set_xlabel(r"SHM rate $\mu$")
ax2 = fig.add_subplot(132)
ax2.set_ylabel("Sensitivity $s$")
ax2.set_xlabel(r"SHM rate $\mu$")
ax3 = fig.add_subplot(133)
ax3.set_ylabel(r"Variation of information $v$")
ax3.set_xlabel(r"SHM rate $\mu$")

colors={"scoper":"forestgreen","hilary-1-1-7_cdr3-based":"crimson","hilary-1-1-7":"k","partis":"gold"}
axes=[ax1,ax2,ax3]

for model in colors:
    print(model)
    for i,metric in enumerate(metric_dict[model]):
        print(metric)
        for freq in metric_dict[model][metric]:
            y=list(metric_dict[model][metric][freq].values())
            x=[freq]*len(y)
            axes[i].plot(
                x, y, "o", fillstyle="none", color=colors[model],
            )
        lengths=[]
        averages=[]
        for freq in metric_dict[model][metric]:
            lengths.append(freq)
            averages.append(np.mean(list(metric_dict[model][metric][freq].values())))
        axes[i].plot(
            lengths,
            averages,
            color=colors[model],
            label=model,
        )

ax1.legend()

plt.tight_layout()
# fig.savefig('new_figures/figure4.pdf',bbox_inches='tight',dpi=1000)
plt.show()
